In [1]:
import numpy as np
import scipy
import scipy.sparse as sp
from scipy.sparse import eye, diags, block_diag, hstack
from scipy.sparse.csgraph import reverse_cuthill_mckee
from scipy.sparse.linalg import gmres
# from scipy.sparse.csgraph import symmetrix_degree_order
import importlib
import time
import NavierStokes_Periodic_Solver as NS_Per

In [3]:
importlib.reload(NS_Per)

<module 'NavierStokes_Periodic_Solver' from '/Users/huynh/Desktop/Python/EnSF/NavierStokes/Periodic/NavierStokes_Periodic/NavierStokes_Periodic_Solver.py'>

## Run on MATLAB first to generate 'Permutation_Indices_RefSol_Per40.mat', then run 'EnSF_DRLM_NS_Periodic' for reference solution

In [5]:
data = scipy.io.loadmat('Permutation_Indices_RefSol_Per40.mat')

# Convert MATLAB arrays to NumPy arrays and cast to float64.
perS = data['perS'].astype(np.float64)
# perA = data['perA'].astype(np.float64)

In [7]:
perS -= 1
# perA -= 1

In [9]:
# Assume the following helper functions are defined:
# -------------------------------
# Domain discretization and time setup
xa = 0 
xb = 1
ya = 0 
yb = 1 
T = 1

mu = 0.001
# n = 8; Nx = 32*n; Ny = Nx; Nt = 8*n;

theta = 5
Nx = 40
Ny = Nx
Nt = T*600

hx = (xb - xa) / Nx
hy = (yb - ya) / Ny
# Create grid points: MATLAB: x = xa:hx:xb, y = ya:hy:yb
x = np.arange(xa, xb + hx/2, hx)  # adding hx/2 ensures xb is included
y = np.arange(ya, yb + hy/2, hy)
xmid = NS_Per.avg(x)
ymid = NS_Per.avg(y)
dt = T / Nt
TT = np.arange(0, T + dt/2, dt)

alpha_BE = 1 / dt   # alpha*u - mu*Delta(u) + grad(p) = f
opt_UgradU = 1   # 1: original, 2: MIT (not good)
opt = 2
# -------------------------------
# Construct matrices A and B

# Sizes:
sU = (Nx) * Ny       # for U-velocity unknowns
sV = sU       # for V-velocity unknowns
sP = sU             # for pressure


# --- Build matrix A ---
A0  = NS_Per.DiscreteLaplace(Nx,hx)
B0  = NS_Per.DiscreteLaplace(Ny,hy)

A_u = alpha_BE * sp.eye(sU) - mu * (sp.kron(sp.eye(Ny), A0) + sp.kron(B0, sp.eye(Nx)))
A_v = A_u.copy()

A_BE = block_diag((A_u, A_v), format='csr')

# # --- Construct matrix B ---

A1 = NS_Per.DiscreteGrad(Nx,hx)         # P_x = A1*P
B1 = NS_Per.DiscreteGrad(Ny,hy)         # P_y = P*B1'

B2 = sp.kron(sp.eye(Ny), A1.T)
B3 = sp.kron(B1.T, sp.eye(Nx))
B = sp.hstack([B2, B3], format='csr')

B = B[1:, :]
Bt = B.transpose().tocsr()

# # --- Prepare matrices for the pressure correction ---
dA_BE = A_BE.diagonal()
D_BE = diags(dA_BE, 0, shape=A_BE.shape, format='csr')
E_BE = D_BE - A_BE
Di_BE = diags(1.0 / dA_BE, 0, shape=A_BE.shape, format='csr')
S_BE = B.dot(Di_BE.dot(Bt))
# # perS = reverse_cuthill_mckee(S)
# # S_perm = S[perS, :][:, perS].toarray()

rowsS, colsS = np.meshgrid(perS, perS)
S_perm = S_BE[rowsS, colsS].toarray()
SS_BE = np.linalg.cholesky(S_perm).T
SS_BEt = SS_BE.T
DiE_BE = Di_BE.dot(E_BE)
BDiE_BE = B.dot(DiE_BE)
DiB_BEt = Di_BE.dot(Bt)

## Create mesh
Yu, Xu = np.meshgrid(ymid, x[0:-1], indexing='xy')

Yv, Xv = np.meshgrid(y[0:-1], xmid, indexing='xy')

Yp, Xp = np.meshgrid(ymid, xmid, indexing='xy')

# Initialize velocity fields using your exact solution functions.
U0 = NS_Per.u_init(Xu, Yu, opt)  # dimensions should match (len(x[1:-1]) x len(ymid))
V0 = NS_Per.v_init(Xv, Yv, opt)

# Initialize pressure-related quantities.
q = 1

q_batch = np.full((2,), q)
# qq = np.zeros(Nt+1)
# qq[0] = q

# egy = np.zeros(Nt + 1)
# egy_theta = egy.copy()
# egy[0] = 0.5 * hx * hy * (NS_Per.inner(U0, U0) + NS_Per.inner(V0, V0))
# egy_theta[0] = egy[0]+theta*(q**2-1)

In [11]:
perSBE_new = perS.astype(int)

perSBE_new = np.squeeze(perSBE_new, axis=0)

In [13]:
data = scipy.io.loadmat('Permutation_Indices_RefSol_BDF2Per40.mat')

# Convert MATLAB arrays to NumPy arrays and cast to float64.
perBDFS = data['perS'].astype(np.float64)

In [15]:
perBDFS -= 1

In [17]:
# Assume the following helper functions are defined:
# -------------------------------
# Domain discretization and time setup
xa = 0 
xb = 1
ya = 0 
yb = 1 
T = 1

mu = 0.001
# n = 8; Nx = 32*n; Ny = Nx; Nt = 8*n;

theta = 5
Nx = 40
Ny = Nx
Nt = T*600

hx = (xb - xa) / Nx
hy = (yb - ya) / Ny
# Create grid points: MATLAB: x = xa:hx:xb, y = ya:hy:yb
x = np.arange(xa, xb + hx/2, hx)  # adding hx/2 ensures xb is included
y = np.arange(ya, yb + hy/2, hy)
xmid = NS_Per.avg(x)
ymid = NS_Per.avg(y)
dt = T / Nt
TT = np.arange(0, T + dt/2, dt)

alpha = 1.5 / dt   # alpha*u - mu*Delta(u) + grad(p) = f
opt_UgradU = 1   # 1: original, 2: MIT (not good)
opt = 2
# -------------------------------
# Construct matrices A and B

# Sizes:
sU = (Nx) * Ny       # for U-velocity unknowns
sV = sU       # for V-velocity unknowns
sP = sU             # for pressure


# --- Build matrix A ---
A0  = NS_Per.DiscreteLaplace(Nx,hx)
B0  = NS_Per.DiscreteLaplace(Ny,hy)

A_u = alpha * sp.eye(sU) - mu * (sp.kron(sp.eye(Ny), A0) + sp.kron(B0, sp.eye(Nx)))
A_v = A_u.copy()

A = block_diag((A_u, A_v), format='csr')

# # --- Construct matrix B ---

A1 = NS_Per.DiscreteGrad(Nx,hx)         # P_x = A1*P
B1 = NS_Per.DiscreteGrad(Ny,hy)         # P_y = P*B1'

B2 = sp.kron(sp.eye(Ny), A1.T)
B3 = sp.kron(B1.T, sp.eye(Nx))
B = sp.hstack([B2, B3], format='csr')

B = B[1:, :]
Bt = B.transpose().tocsr()

# # --- Prepare matrices for the pressure correction ---
dA = A.diagonal()
D = diags(dA, 0, shape=A.shape, format='csr')
E = D - A
Di = diags(1.0 / dA, 0, shape=A.shape, format='csr')
S = B.dot(Di.dot(Bt))
# # perS = reverse_cuthill_mckee(S)
# # S_perm = S[perS, :][:, perS].toarray()

rowsS, colsS = np.meshgrid(perBDFS, perBDFS)
S_perm = S[rowsS, colsS].toarray()
SS = np.linalg.cholesky(S_perm).T
SSt = SS.T
DiE = Di.dot(E)
BDiE = B.dot(DiE)
DiBt = Di.dot(Bt)

## Create mesh
Yu, Xu = np.meshgrid(ymid, x[0:-1], indexing='xy')

Yv, Xv = np.meshgrid(y[0:-1], xmid, indexing='xy')

Yp, Xp = np.meshgrid(ymid, xmid, indexing='xy')

# Initialize velocity fields using your exact solution functions.
U0 = NS_Per.u_init(Xu, Yu, opt)  # dimensions should match (len(x[1:-1]) x len(ymid))
V0 = NS_Per.v_init(Xv, Yv, opt)

# Initialize pressure-related quantities.
q = 1

# q_batch = np.full((2,), q)
# qq = np.zeros(Nt+1)
# qq[0] = q

# egy = np.zeros(Nt + 1)
# egy_theta = egy.copy()
# egy[0] = 0.5 * hx * hy * (NS_Per.inner(U0, U0) + NS_Per.inner(V0, V0))
# egy_theta[0] = egy[0]+theta*(q**2-1)

In [18]:
perBDFS_new = perBDFS.astype(int)

perBDFS_new = np.squeeze(perBDFS_new, axis=0)

In [20]:
mU0, nU0 = U0.shape
mV0, nV0 = V0.shape
mP0, nP0 = Xp.shape

In [23]:
All_U = np.zeros((Nt+1, mU0*nU0))
All_V = np.zeros((Nt+1, mV0*nV0))
All_P = np.zeros((Nt+1, mP0*nP0))
All_q = np.zeros((Nt+1, 1))

In [25]:
All_U[0, :] += np.reshape(U0, mU0*nU0, order='F')
All_V[0, :] += np.reshape(V0, mV0*nV0, order='F')
All_q[0, :] += 1

In [27]:
Size_U = mU0*nU0
Size_V = mV0*nV0
Size_P = mP0*nP0

In [29]:
def local_indices(p, Nx, Ny, r):
    """
    Return the flattened indices of a square neighborhood of radius r
    around grid point p on an Nx-by-Ny grid (0-based indexing).

    Parameters:
      p  : integer in [0, Nx*Ny)
      Nx : number of rows
      Ny : number of columns
      r  : localization radius (in grid cells)

    Returns:
      flat_inds : 1D array of length up to (2r+1)^2, listing all
                  0-based flattened indices within the neighborhood.
    """
    # Convert flat index p -> (i,j)
    i = p // Ny
    j = p % Ny
    # Row range [i-r, i+r], clipped to [0, Nx)
    rows = np.arange(max(0, i-r), min(Nx, i+r+1))
    # Col range [j-r, j+r], clipped to [0, Ny)
    cols = np.arange(max(0, j-r), min(Ny, j+r+1))
    # Form Cartesian product
    Ii, Jj = np.meshgrid(rows, cols, indexing='ij')
    flat_inds = (Ii * Ny + Jj).ravel()
    return flat_inds

In [31]:
def local_indices_block(p, Nx, Ny, radius):
    ## Modify this
    m = Nx * Ny
    # determine component: 0,1,2
    comp = p // m
    # pixel idx within component
    q = p % m
    # local patch in component
    base_patch = local_indices(q, Nx, Ny, radius)
    # shift by component block
    return base_patch + comp * m

In [33]:
def local_indices_block_diffsize(p, Nx, Ny, radius):
    """
    Return global indices of the neighborhood around global index p,
    where the state is composed of 4 blocks of sizes:
      1) Nx*Ny       on an (Nx,Ny) grid
      2) Nx*(Ny-1)   on an (Nx,Ny-1) grid
      3) Nx*Ny       on an (Nx,Ny) grid
      4) 1           (scalar)
    """
    # 1) block sizes
    m1 = Nx * Ny
    m2 = Nx * Ny
    m3 = Nx * Ny
    m4 = 1
    block_sizes = [m1, m2, m3, m4]
    # 2) compute block offsets
    offsets = np.array([0, m1, m1 + m2, m1 + m2 + m3], dtype=int)

    # 3) find which block p is in
    for comp, size in enumerate(block_sizes):
        if p < offsets[comp] + size:
            break

    # 4) local index within that block
    q = p - offsets[comp]

    # 5) build the neighborhood
    if comp == 0 or comp == 2:
        # Blocks 0 & 2: Nx x Ny grid
        base_patch = local_indices(q, Nx, Ny, radius)
    elif comp == 1:
        # Block 1: Nx x (Ny-1) grid
        base_patch = local_indices(q, Nx, Ny, radius)
    else:
        # Block 3: scalar
        base_patch = np.array([0], dtype=int)

    # 6) shift to global indices
    return base_patch + offsets[comp]

In [35]:
def nearest_k_obs(i, grid, obs_xy, scalar_idx, K):
    
    if i == scalar_idx:
        return np.array([-1], dtype=int)
    else:
        d2 = ((obs_xy - grid[i])**2).sum(axis=1)
        idx = np.argsort(d2)[:K]
        return idx

In [37]:
def LETKF_local(Xb, y, R, rho, Nx, Ny, idx_obs, grid_xy_total, obs_xy, radius):
    """
    Local LETKF using a square neighborhood in model space (no obs localization).

    Steps 1–2: global means and perturbations
    Step 3  : local model-space selection via `local_indices`
    Steps 4–8: analysis in ensemble space (obs from all dims)

    Inputs:
      Xb     : (m_g, k) forecast ensemble (m_g = Nx*Ny)
      y      : (l_g,)    observations
      H      : (l_g, m_g) observation operator
      R      : (l_g, l_g) observation error covariance
      Nx, Ny : grid dimensions
      radius : localization radius in grid cells

    Returns:
      Xa : (m_g, k) analysis ensemble
    """
    m_g, k = Xb.shape
    # print(m_g)
    # Step 1 & 2: global means & perturbations
    Xb_mean = Xb.mean(axis=1, keepdims=True)      # (m_g, 1)
    Xb_pert = Xb - Xb_mean                        # (m_g, k)
    Yb_raw  = np.arctan(Xb[idx_obs, :])          # (l_g, k)                
    Yb_mean = Yb_raw.mean(axis=1, keepdims=True)  # (l_g, 1)
    Yb_pert = Yb_raw - Yb_mean                    # (l_g, k)

    Xa = np.zeros_like(Xb)

    # Precompute obs-space inverse
#     Rinv = np.linalg.inv(R)

    # Loop over each grid point for local analysis
    for p in range(m_g):
        # Step 3: pick local model-space indices
        loc_inds = local_indices_block_diffsize(p, Nx, Ny, radius)
        # loc_inds = local_indices_block(p, Nx, Ny, radius)
        # print(p)
        # Local mean & perturbations
        xb_m_loc = Xb_mean[loc_inds, 0]           # (m_loc,)
        Xb_loc   = Xb_pert[loc_inds, :]           # (m_loc, k)

        # Step 4: use all observations (no spatial selection)
        loc_obs = nearest_k_obs(p, grid_xy_total, obs_xy, m_g-1, 8)
        # if p == m_g-1:
        #     print(loc_obs)
        yb_m_loc = Yb_mean[loc_obs, 0]                  # (l_g,)
        Yb_loc   = Yb_pert[loc_obs, :]                        # (l_g, k)
      
        yo_loc = y[loc_obs, 0] # observation
        R_loc = R[loc_obs]           # now shape (n_obs, l_g)
        R_loc = R_loc[:, loc_obs] 
        # Steps 5–6: ensemble-space analysis covariance
        W = np.linalg.solve(R_loc, Yb_loc)
        C = W.T 

        # Step 7a: analysis perturbation matrix
        M = (k-1)/rho * np.eye(k) + C @ Yb_loc
        w, Q = np.linalg.eigh(M)
        w_inv     = 1.0 / w          # for P^a = Q diag(w_inv) Q^T
        w_inv_sqrt = 1.0 / np.sqrt(w)  # for (P^a)^{1/2}
        
        Pa = (Q * w_inv.reshape(1, -1)) @ Q.T 

        # Step 7b: mean weights
        Wa = np.sqrt(k-1) * (Q * w_inv_sqrt.reshape(-1, 1)) @ Q.T
        wabar = Pa @ (C @ (yo_loc - yb_m_loc))           # (k,)
        Wana  = Wa + wabar.reshape(-1, 1)                 # (k x k)
        
        # Step 8: map back to model space and extract center
        xa_loc = xb_m_loc[:, None] + Xb_loc @ Wana  # (m_loc, k)
        # find center index within the local block
        
        center_idx = np.where(loc_inds == p)[0][0]
        Xa[p, :] = xa_loc[center_idx, :]

    return Xa

In [39]:
# ## 100% arctangent observation
# indices_U = np.random.permutation(Size_U)[:int(1 * Size_U)]
# indices_V = np.random.permutation(Size_V)[:int(1 * Size_V)] + Size_U
# indices_P = np.random.permutation(Size_P)[:int(1 * Size_P)] + Size_U + Size_V
# indices_q = np.array(Size_U+Size_V+Size_P)


## 70% arctangent observation
# indices_U = np.random.permutation(Size_U)[:int(0.7 * Size_U)]
# indices_V = np.random.permutation(Size_V)[:int(0.7 * Size_V)] + Size_U
# indices_P = np.random.permutation(Size_P)[:int(0.7 * Size_P)] + Size_U + Size_V
# indices_q = np.array(Size_U+Size_V+Size_P)


## 7% arctangent observation
indices_U = np.random.permutation(Size_U)[:int(0.07* Size_U)]
indices_V = np.random.permutation(Size_V)[:int(0.07* Size_V)] + Size_U
indices_P = np.random.permutation(Size_P)[:int(0.07* Size_P)] + Size_U + Size_V
indices_q = np.array(Size_U+Size_V+Size_P)
num_indices= indices_U.size+indices_V.size+indices_P.size+1

# Combine and sort the selected indices
spa_indices = np.sort(np.concatenate([indices_U, indices_V, indices_P, indices_q.reshape(1)]))

337
4800


In [41]:
data1 = scipy.io.loadmat('TestRefSol_BDF2_Periodic_v2.mat')
U_Ref = data1['U_Py']
V_Ref = data1['V_Py']
P_Ref = data1['P_Py']
q_Ref = data1['q_Py']

In [43]:
# ntEnSF = 50
ntEnSF = 100
t0 = 0
filtering_steps = ntEnSF
timeTrue = np.linspace(0, 1, Nt+1)
tEnSF = np.linspace(0, 1, filtering_steps+1)
indices_time = np.searchsorted(timeTrue, tEnSF, side='left')


state_ref = np.concatenate((U_Ref, V_Ref, P_Ref, q_Ref), axis=1)   

state_timeextract = state_ref[indices_time, :].copy()

state_EnSF = state_timeextract[:, spa_indices].copy()

dtEnSF = (T - t0) / ntEnSF
obs_sigma = 0.1

eps_alpha = 0.05

# ensemble size
ensemble_size = 80
ensemble_true = 1
# forward Euler step
euler_steps = 400
def g_tau(t):
    return 1-t

U0_state = 2*np.random.randn(ensemble_size, Size_U)
V0_state = 2*np.random.randn(ensemble_size, Size_V)
P0_state = 2*np.random.randn(ensemble_size, Size_P)

UV_state = np.concatenate((U0_state, V0_state, P0_state, np.full((ensemble_size, 1), 1)), axis=1)

n_dim = Size_U+Size_V+Size_P+1
rmse_all = []
obs_save = []
est_save = np.zeros((filtering_steps+1, n_dim))
est_save[[0], :] += np.mean(UV_state, axis=0)

(601, 4801)


In [45]:
## Noise omega_1
SDE_Sigma_U = 0.001
SDE_Sigma_V = 0.001
SDE_Sigma_P = 0.001

## Noise omega_1
# SDE_Sigma_U = 0.1
# SDE_Sigma_V = 0.1
# SDE_Sigma_P = 0.1

In [47]:
data1 = scipy.io.loadmat('Permutation_Indices_EnSF_Per40_T100.mat')
# data1 = scipy.io.loadmat('Permutation_Indices_EnSF_Per40.mat')
# Convert MATLAB arrays to NumPy arrays and cast to float64.
perS_EnSF = data1['perS'].astype(np.float64)

In [49]:
perS_EnSF -= 1

In [51]:
# Assume the following helper functions are defined:
# -------------------------------
# Domain discretization and time setup
xa = 0 
xb = 1
ya = 0 
yb = 1 
T = 1

mu = 0.001
# n = 8; Nx = 32*n; Ny = Nx; Nt = 8*n;

theta = 5
Nx = 40
Ny = Nx
# Nt = T*50

hx = (xb - xa) / Nx
hy = (yb - ya) / Ny
# Create grid points: MATLAB: x = xa:hx:xb, y = ya:hy:yb
x = np.arange(xa, xb + hx/2, hx)  # adding hx/2 ensures xb is included
y = np.arange(ya, yb + hy/2, hy)
xmid = NS_Per.avg(x)
ymid = NS_Per.avg(y)
# dt = T / Nt
TTEnSF = np.arange(0, T + dtEnSF/2, dtEnSF)

alpha_EnSFBE = 1 / dtEnSF   # alpha*u - mu*Delta(u) + grad(p) = f
opt_UgradU = 1   # 1: original, 2: MIT (not good)
opt = 2
# -------------------------------
# Construct matrices A and B

# Sizes:
sU = (Nx) * Ny       # for U-velocity unknowns
sV = sU       # for V-velocity unknowns
sP = sU             # for pressure


# --- Build matrix A ---
A0  = NS_Per.DiscreteLaplace(Nx,hx)
B0  = NS_Per.DiscreteLaplace(Ny,hy)

A_u = alpha_EnSFBE * sp.eye(sU) - mu * (sp.kron(sp.eye(Ny), A0) + sp.kron(B0, sp.eye(Nx)))
A_v = A_u.copy()

A_EnSFBE = block_diag((A_u, A_v), format='csr')

# # --- Construct matrix B ---

A1 = NS_Per.DiscreteGrad(Nx,hx)         # P_x = A1*P
B1 = NS_Per.DiscreteGrad(Ny,hy)         # P_y = P*B1'

B2 = sp.kron(sp.eye(Ny), A1.T)
B3 = sp.kron(B1.T, sp.eye(Nx))
B = sp.hstack([B2, B3], format='csr')

B = B[1:, :]
Bt = B.transpose().tocsr()

# # --- Prepare matrices for the pressure correction ---
dA_EnSFBE = A_EnSFBE.diagonal()
D_EnSFBE = diags(dA_EnSFBE, 0, shape=A_EnSFBE.shape, format='csr')
E_EnSFBE = D_EnSFBE - A_EnSFBE
Di_EnSFBE = diags(1.0 / dA_EnSFBE, 0, shape=A_EnSFBE.shape, format='csr')
S_EnSFBE = B.dot(Di_EnSFBE.dot(Bt))
# # perS = reverse_cuthill_mckee(S)
# # S_perm = S[perS, :][:, perS].toarray()

rowsS, colsS = np.meshgrid(perS_EnSF, perS_EnSF)
S_perm = S_EnSFBE[rowsS, colsS].toarray()
SS_EnSFBE = np.linalg.cholesky(S_perm).T
SS_EnSFBEt = SS_EnSFBE.T
DiE_EnSFBE = Di_EnSFBE.dot(E_EnSFBE)
BDiE_EnSFBE = B.dot(DiE_EnSFBE)
DiB_EnSFBEt = Di_EnSFBE.dot(Bt)

## Create mesh
Yu, Xu = np.meshgrid(ymid, x[0:-1], indexing='xy')

Yv, Xv = np.meshgrid(y[0:-1], xmid, indexing='xy')

Yp, Xp = np.meshgrid(ymid, xmid, indexing='xy')

# Initialize velocity fields using your exact solution functions.
# U0 = NS_Per.u_init(Xu, Yu, opt)  # dimensions should match (len(x[1:-1]) x len(ymid))
# V0 = NS_Per.v_init(Xv, Yv, opt)

# Initialize pressure-related quantities.
q = 1

q_batch = np.full((ensemble_size,), q)
# qq = np.zeros(Nt+1)
# qq[0] = q

# egy = np.zeros(Nt + 1)
# egy_theta = egy.copy()
# egy[0] = 0.5 * hx * hy * (NS_Per.inner(U0, U0) + NS_Per.inner(V0, V0))
# egy_theta[0] = egy[0]+theta*(q**2-1)

In [53]:
perS_EnSFnew = perS_EnSF.astype(int)

perS_EnSFnew = np.squeeze(perS_EnSFnew, axis=0)

In [55]:
grid_U = np.stack([(Xu.T).reshape(-1), (Yu.T).reshape(-1)], axis=1)
grid_V = np.stack([(Xv.T).reshape(-1), (Yv.T).reshape(-1)], axis=1)
grid_P = np.stack([(Xp.T).reshape(-1), (Yp.T).reshape(-1)], axis=1)

grid_total = np.vstack((grid_U, grid_V, grid_P))
obs_xy    = grid_total[spa_indices[:-1]]

In [61]:
for i in range(filtering_steps):
    print(f'step={i}:')
    t1 = time.time()    
    
#     obs = state_EnSF[[i+1], :].copy()
    state_scale = state_EnSF[[i+1], :].copy()
    
    indob_scale0 = np.nonzero(((-1e-1 <= state_scale) & (state_scale < -1e-2)) |
                              ((1e-2 <= state_scale) & (state_scale < 1e-1)))[1]
    
    indob_scale1 = np.nonzero(((-1e-2 <= state_scale) & (state_scale < -1e-3)) |
                              ((1e-3 <= state_scale) & (state_scale < 1e-2)))[1]
    
    indob_scale2 = np.nonzero(((-1e-3 <= state_scale) & (state_scale < -1e-4)) |
                              ((1e-4 <= state_scale) & (state_scale < 1e-3)))[1]

    indob_scale3 = np.nonzero(((-1e-4 <= state_scale) & (state_scale < -1e-5)) |
                              ((1e-5 <= state_scale) & (state_scale < 1e-4)))[1]

    indob_scale4 = np.nonzero(((-1e-5 <= state_scale) & (state_scale < -1e-6)) |
                              ((1e-6 <= state_scale) & (state_scale < 1e-5)))[1]
    indob_scale5 = np.nonzero(((-1e-6 <= state_scale) & (state_scale < -1e-7)) |
                              ((1e-7 <= state_scale) & (state_scale < 1e-6)))[1]
    
    indob_scale6 = np.nonzero(((-1e-12 <= state_scale) & (state_scale < -1e-13)) |
                              ((1e-13 <= state_scale) & (state_scale < 1e-12)))[1]

    indob_scale7 = np.nonzero(((-1e-13 <= state_scale) & (state_scale < -1e-14)) |
                              ((1e-14 <= state_scale) & (state_scale < 1e-13)))[1]
    
    indob_scale8 = np.nonzero(((-1e-14 <= state_scale) & (state_scale < -1e-15)) |
                              ((1e-15 <= state_scale) & (state_scale < 1e-14)))[1]
    
    indob_scale9 = np.nonzero(((-1e-15 <= state_scale) & (state_scale < -1e-16)) |
                              ((1e-16 <= state_scale) & (state_scale < 1e-15)))[1]

    indob_scale10 = np.nonzero(((-1e-16 <= state_scale) & (state_scale < -1e-17)) |
                              ((1e-17 <= state_scale) & (state_scale < 1e-16)))[1]
    indob_scale11 = np.nonzero(((-1e-17 <= state_scale) & (state_scale < 0)) |
                              ((0 <= state_scale) & (state_scale < 1e-17)))[1]
    
    state_scale[:, indob_scale0] *= 1e1
    state_scale[:, indob_scale1] *= 1e2
    state_scale[:, indob_scale2] *= 1e3
    state_scale[:, indob_scale3] *= 1e4
    state_scale[:, indob_scale4] *= 1e5
    state_scale[:, indob_scale5] *= 1e6
    state_scale[:, indob_scale6] *= 1e12
    state_scale[:, indob_scale7] *= 1e13
    state_scale[:, indob_scale8] *= 1e14
    state_scale[:, indob_scale9] *= 1e15
    state_scale[:, indob_scale10] *= 1e16
    state_scale[:, indob_scale11] *= 1e17
    
    obs = np.arctan(state_scale.copy())
    obs += np.random.randn(*state_EnSF[[i+1], :].shape) * obs_sigma

#     def score_likelihood(xt, t, C):
#         # obs: (d)
#         # xt: (ensemble, d)
# #         A = -(xt - obs) / obs_sigma**2
# #         score_x = A.copy()
# #         score_x[:, idA_sub] =\
# #             -(np.arctan(xt[:, idA_sub]) - obs[:, idA_sub]) / obs_sigma**2 * (1. / (1 + xt[:, idA_sub]**2))
#         score_x = -(np.arctan(xt) - obs)/obs_sigma**2 * (1./(1+xt**2))
#         tau = g_tau(t)
#         return tau * score_x / C
       
    U_stack = (U0_state.T).copy()
    V_stack = (V0_state.T).copy()
    
    U_stack = U_stack.reshape(mU0, nU0, ensemble_size, order = 'F')
    V_stack = V_stack.reshape(mV0, nV0, ensemble_size, order = 'F')
    
    U_new, V_new, P_new, q_new, egy_new, egy_theta_new,_ = \
            NS_Per.NS_BE_1step_Periodic_Vectorized(hx, hy, dtEnSF, TTEnSF[i+1], U_stack, V_stack, q_batch, Xu, Yu, Xv, Yv,\
                                                   mu, theta, opt, opt_UgradU, DiE_EnSFBE, BDiE_EnSFBE, DiB_EnSFBEt,\
                                                   Di_EnSFBE, B, Bt, perS_EnSFnew, SS_EnSFBE, SS_EnSFBEt, Nx, Ny, sU,\
                                                   alpha_EnSFBE, A1, B1)
    
    U_new_reshape = U_new.reshape(mU0*nU0, ensemble_size, order ='F')  
    V_new_reshape = V_new.reshape(mV0*nV0, ensemble_size, order ='F')
    P_new_reshape = P_new.reshape(mP0*nP0, ensemble_size, order ='F')
    q_new_reshape = q_new.reshape(1, ensemble_size)
    
    # q_batch = q_new.copy()
    x_state = np.concatenate((U_new_reshape, V_new_reshape, P_new_reshape, q_new_reshape))
    
    noiseU = np.sqrt(dtEnSF) * SDE_Sigma_U * np.random.randn(*U_new_reshape.shape)
    noiseV = np.sqrt(dtEnSF) * SDE_Sigma_V * np.random.randn(*V_new_reshape.shape)
    noiseP = np.sqrt(dtEnSF) * SDE_Sigma_P * np.random.randn(*P_new_reshape.shape)

    noise = np.concatenate((noiseU, noiseV, noiseP, np.zeros((1, ensemble_size))))
    
    x_state += noise
    x_state = x_state.T
    
    x0_EnSF = x_state[:, spa_indices].copy()
    
    R = obs_sigma**2 * np.eye(spa_indices.shape[0])
    
    indx_scale0 = np.argwhere(((-1e-1<=x0_EnSF) & (x0_EnSF<-1e-2)) | ((1e-2<=x0_EnSF) & (x0_EnSF<1e-1)))
    indx_scale1 = np.argwhere(((-1e-2<=x0_EnSF) & (x0_EnSF<-1e-3)) | ((1e-3<=x0_EnSF) & (x0_EnSF<1e-2)))
    indx_scale2 = np.argwhere(((-1e-3<=x0_EnSF) & (x0_EnSF<-1e-4)) | ((1e-4<=x0_EnSF) & (x0_EnSF<1e-3)))
    indx_scale3 = np.argwhere(((-1e-4<=x0_EnSF) & (x0_EnSF<-1e-5)) | ((1e-5<=x0_EnSF) & (x0_EnSF<1e-4)))
    indx_scale4 = np.argwhere(((-1e-5<=x0_EnSF) & (x0_EnSF<-1e-6)) | ((1e-6<=x0_EnSF) & (x0_EnSF<1e-5)))
    indx_scale5 = np.argwhere(((-1e-6<=x0_EnSF) & (x0_EnSF<-1e-7)) | ((1e-7<=x0_EnSF) & (x0_EnSF<1e-6)))
    indx_scale6 = np.argwhere(((-1e-12<=x0_EnSF) & (x0_EnSF<-1e-13)) | ((1e-13<=x0_EnSF) & (x0_EnSF<1e-12)))
    indx_scale7 = np.argwhere(((-1e-13<=x0_EnSF) & (x0_EnSF<-1e-14)) | ((1e-14<=x0_EnSF) & (x0_EnSF<1e-13)))
    indx_scale8 = np.argwhere(((-1e-14<=x0_EnSF) & (x0_EnSF<-1e-15)) | ((1e-15<=x0_EnSF) & (x0_EnSF<1e-14)))
    indx_scale9 = np.argwhere(((-1e-15<=x0_EnSF) & (x0_EnSF<-1e-16)) | ((1e-16<=x0_EnSF) & (x0_EnSF<1e-15)))
    indx_scale10 = np.argwhere(((-1e-16<=x0_EnSF) & (x0_EnSF<-1e-17)) | ((1e-17<=x0_EnSF) & (x0_EnSF<1e-16)))
    indx_scale11 = np.argwhere(((-1e-17<=x0_EnSF) & (x0_EnSF<0)) | ((0<=x0_EnSF) & (x0_EnSF<1e-17)))

    x0_EnSF[indx_scale0[:, 0], indx_scale0[:, 1]] *= 1e1
    x0_EnSF[indx_scale1[:, 0], indx_scale1[:, 1]] *= 1e2
    x0_EnSF[indx_scale2[:, 0], indx_scale2[:, 1]] *= 1e3
    x0_EnSF[indx_scale3[:, 0], indx_scale3[:, 1]] *= 1e4
    x0_EnSF[indx_scale4[:, 0], indx_scale4[:, 1]] *= 1e5
    x0_EnSF[indx_scale5[:, 0], indx_scale5[:, 1]] *= 1e6
    x0_EnSF[indx_scale6[:, 0], indx_scale6[:, 1]] *= 1e12
    x0_EnSF[indx_scale7[:, 0], indx_scale7[:, 1]] *= 1e13
    x0_EnSF[indx_scale8[:, 0], indx_scale8[:, 1]] *= 1e14
    x0_EnSF[indx_scale9[:, 0], indx_scale9[:, 1]] *= 1e15
    x0_EnSF[indx_scale10[:, 0], indx_scale10[:, 1]] *= 1e16
    x0_EnSF[indx_scale11[:, 0], indx_scale11[:, 1]] *= 1e17
    
    x_state[:, spa_indices] = x0_EnSF.copy()
#     Xb_LETKF = x_state[:, spa_indices].clone()
    Xb_LETKF = x_state.copy()
    Xb = Xb_LETKF.T
    y  = (obs.copy()).T
#     rho = 2
    rho = 4
    radius = 1
    Xa_LETKF = LETKF_local(Xb, y, R, rho, Nx, Ny, spa_indices,  grid_total, obs_xy, radius)
    x_scale = (Xa_LETKF.copy()).T
    x0_EnSF = x_scale[:, spa_indices].copy()

    x0_EnSF[:, indob_scale0] /= 1e1
    x0_EnSF[:, indob_scale1] /= 1e2
    x0_EnSF[:, indob_scale2] /= 1e3
    x0_EnSF[:, indob_scale3] /= 1e4
    x0_EnSF[:, indob_scale4] /= 1e5
    x0_EnSF[:, indob_scale5] /= 1e6
    x0_EnSF[:, indob_scale6] /= 1e12
    x0_EnSF[:, indob_scale7] /= 1e13
    x0_EnSF[:, indob_scale8] /= 1e14
    x0_EnSF[:, indob_scale9] /= 1e15
    x0_EnSF[:, indob_scale10] /= 1e16
    x0_EnSF[:, indob_scale11] /= 1e17

    x_scale[:, spa_indices] = x0_EnSF.copy()
    x_state = x_scale.copy()
    
    x_state[:, np.arange(0, Size_U)] = np.clip(x_state[:, np.arange(0, Size_U)], -0.9, 0.9)
    x_state[:, Size_U+np.arange(0, Size_V)] = np.clip(x_state[:, Size_U+np.arange(0, Size_V)], -0.9, 0.9)
    x_state[:, Size_U+Size_V+np.arange(0, Size_P)] = np.clip(x_state[:, Size_U+Size_V+np.arange(0, Size_P)], -0.6, 0.6)
    x_state[:, -1] = np.clip(x_state[:, -1], 0.8, 1.05)
    
    U0_state = x_state[:, :Size_U].copy()
    V0_state = x_state[:, Size_U+np.arange(0, Size_V)].copy()
    q_batch = x_state[:, -1].copy()
    print(q_Ref[i+1])
    # print(q_batch)
    est_save[[i+1], :] += np.mean(x_state, axis=0)
    print(est_save[i+1, -1])
    # get rmse
    rmse_temp = np.sqrt(np.mean((est_save[[i+1], :] - state_timeextract[[i+1], :])**2))

    # get time
    t2 = time.time()
    print(f'\t RMSE = {rmse_temp:.4f}')
    print(f'\t time = {t2 - t1:.4f}')

    # save information
    rmse_all.append(rmse_temp)

    # check divergence
    if rmse_temp > 1000:
        print('diverge!')
        break

step=0:
(80, 4801)
(4801, 80)
[1.]
1.0500000000000003
	 RMSE = 0.4288
	 time = 64.3215
step=1:
(80, 4801)
(4801, 80)
[1.]
1.0500000000000003
	 RMSE = 0.4217
	 time = 77.6973
step=2:
(80, 4801)
(4801, 80)
[1.]
1.0500000000000003
	 RMSE = 0.4149
	 time = 64.5313
step=3:
(80, 4801)
(4801, 80)
[1.]
1.049994840885979
	 RMSE = 0.4103
	 time = 65.8883
step=4:
(80, 4801)
(4801, 80)
[1.]
1.0499646508038807
	 RMSE = 0.4071
	 time = 64.5166
step=5:
(80, 4801)
(4801, 80)
[1.]
1.0499065141804533
	 RMSE = 0.4047
	 time = 64.8894
step=6:
(80, 4801)
(4801, 80)
[1.]
1.049812924261453
	 RMSE = 0.4029
	 time = 64.4247
step=7:
(80, 4801)
(4801, 80)
[1.]
1.0496546413636316
	 RMSE = 0.4006
	 time = 63.8268
step=8:
(80, 4801)
(4801, 80)
[1.]
1.0493697302734049
	 RMSE = 0.3988
	 time = 68.3709
step=9:
(80, 4801)
(4801, 80)
[1.]
1.0488430142500291
	 RMSE = 0.3967
	 time = 193.5091
step=10:
(80, 4801)
(4801, 80)
[1.]
1.0478675158605382
	 RMSE = 0.3952
	 time = 65.7030
step=11:
(80, 4801)
(4801, 80)
[1.]
1.04601

In [63]:
U_EnSF = est_save[:, :Size_U]
V_EnSF = est_save[:, Size_U+np.arange(0, Size_V)]
P_EnSF = est_save[:, Size_U+Size_V+np.arange(0, Size_P)]
q_EnSF = est_save[:, -1]

In [65]:
scipy.io.savemat('ResultLETKF_Periodic_T100_70Obs_noise0001_v1.mat', {'U_EnSF':U_EnSF, 'V_EnSF':V_EnSF, 'P_EnSF':P_EnSF, \
                                               'q_EnSF': q_EnSF, 'rmse': rmse_all})

# scipy.io.savemat('ResultLETKF_Periodic_T100_7Obs_noise01_v1.mat', {'U_EnSF':U_EnSF, 'V_EnSF':V_EnSF, 'P_EnSF':P_EnSF, \
#                                                'q_EnSF': q_EnSF, 'rmse': rmse_all})